In [14]:
from dotenv import load_dotenv

load_dotenv()  # .env 파일을 환경 변수로 로드

True

In [22]:
"""
#5.6 Chat Based Memory (04:03)

Memory 클래스는 문자열, message 형태로 출력 할 수 있음.
MessagesPlaceholder 는 예측하기 어려운 메세지의 양과 제한 없는 양의 메세지를 가질 수 있음
"""
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate,MessagesPlaceholder

In [23]:
llm = ChatOpenAI(temperature=0.1)
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True, # 메세지 형태로 출력
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)

chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=prompt,
    verbose=True # chain의 프롬프트 로그 확인 가능
)

In [24]:
chain.predict(question="My name is Dooseop")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Dooseop

> Finished chain.


'Hello Dooseop! How can I assist you today?'

In [25]:
chain.predict(question="I live in Seoul")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Dooseop
AI: Hello Dooseop! How can I assist you today?
Human: I live in Seoul

> Finished chain.


'Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or discuss about Seoul?'

In [26]:
chain.predict(question="What is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful AI talking to a human
Human: My name is Dooseop
AI: Hello Dooseop! How can I assist you today?
Human: I live in Seoul
AI: Seoul is a vibrant city with a rich history and culture. Is there anything specific you would like to know or discuss about Seoul?
Human: What is my name?

> Finished chain.


'Your name is Dooseop. How can I assist you further, Dooseop?'